<a href="https://colab.research.google.com/github/NoeGille/UNet-on-fashion-mnist/blob/main/UNet_on_fashion_MNIST_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

In [19]:
in_channels = 1
batch_size = 64
learning_rate = 0.001
num_classes = 10
epochs = 1


train_dataset = datasets.FashionMNIST(root='tmp/dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.FashionMNIST(root='tmp/dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
class DownSampleBlock(nn.Module):
    '''Reduce the dimension of the image in input by 2'''
    def __init__(self, in_channels):
        super().__init__()
        # We keep the same dimension in input and ouput
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1,1))
        self.relu = nn.ReLu()
        self.conv2 = nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2,2))

    def forward(self, x):
        x = self.pool(self.conv2(self.relu(self.conv1(x))))
        return x

class UNet(nn.Module):
    def __init__(self, input_size, num_classes:int=10, depth:int=2):
        '''### Initialize a UNet model
        input_size : dimension of input
        num_classes : specify the number of classes in ouput
        depth : the number of blocks (depth of the model)'''
        super().__init__()
        # Reduce size of inputs
        dblocks = []
        self.bottleneck = Bottleneck()
        # Increase size of inputs
        ublocks = []
        for i in range(depth):
            dblocks.append(DownSampleBlock())





In [26]:
def transform_targets(X, y_label):

    y = np.array([cv2.threshold(img, 8, 255, type=cv2.THRESH_BINARY)[1] for img in X.numpy()])
    y = np.where(y == 0, 0, 1)
    for s in range(y.shape[0]):
      y[s] *= y_label[s].numpy() + 1
    return torch.from_numpy(np.eye(num_classes, dtype='uint8')[y])


for data,y_label in tqdm(train_loader):
  targets = transform_targets(data, y_label)

  data.to(device=device)
  

100%|██████████| 938/938 [00:12<00:00, 72.96it/s]
